<a href="https://colab.research.google.com/github/Toshiki18/mesh_2d_map/blob/main/mesh_map_sample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install jismeshcode

In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import json

In [ ]:
# メッシュコード、スコア、アドレスのリスト
df = [
    {"meshcode": "53394501", "score": 10, "address": "Address 1"},
    {"meshcode": "53394502", "score": 20, "address": "Address 2"},
    {"meshcode": "53394503", "score": 30, "address": "Address 3"},
    {"meshcode": "53394504", "score": 30, "address": "Address 3"}
]
df = pd.DataFrame(df)

In [ ]:
df['meshcode'].astype(int)

0    53394501
1    53394502
2    53394503
3    53394504
Name: meshcode, dtype: int64

In [ ]:
from jismeshcode.jismeshcode import mc2geojson


In [ ]:
def polygon_geo(polygon):
  return json.dumps(polygon['geometry'])

In [ ]:
df["polygon"] = df['meshcode'].apply(mc2geojson)

In [ ]:
df.head()

,meshcode,score,address,polygon
0,53394501,10,Address 1,"{""type"": ""Feature"", ""geometry"": {""type"": ""Poly..."
1,53394502,20,Address 2,"{""type"": ""Feature"", ""geometry"": {""type"": ""Poly..."
2,53394503,30,Address 3,"{""type"": ""Feature"", ""geometry"": {""type"": ""Poly..."
3,53394504,30,Address 3,"{""type"": ""Feature"", ""geometry"": {""type"": ""Poly..."


In [ ]:
# GeoJSON形式に変換
import json
from shapely.geometry import shape

def extract_coordinates(polygon):
    geom = json.loads(polygon)['geometry']
    return geom['coordinates'][0]

df['coordinates'] = df['polygon'].apply(extract_coordinates)
df.head()

,meshcode,score,address,polygon,coordinates
0,53394501,10,Address 1,"{""type"": ""Feature"", ""geometry"": {""type"": ""Poly...","[[139.6375, 35.66666666666667], [139.649999999..."
1,53394502,20,Address 2,"{""type"": ""Feature"", ""geometry"": {""type"": ""Poly...","[[139.65, 35.66666666666667], [139.6625, 35.66..."
2,53394503,30,Address 3,"{""type"": ""Feature"", ""geometry"": {""type"": ""Poly...","[[139.6625, 35.66666666666667], [139.674999999..."
3,53394504,30,Address 3,"{""type"": ""Feature"", ""geometry"": {""type"": ""Poly...","[[139.675, 35.66666666666667], [139.6875, 35.6..."


In [ ]:
# まとめたGeoJSONの初期化
combined_geojson = {
    "type": "FeatureCollection",
    "features": []
}

# 各行のpolygon列を解析してcombined_geojsonのfeaturesに追加
for index, row in df.iterrows():
    polygon = json.loads(row['polygon'])
    combined_geojson["features"].append(polygon)

# 結果をJSON文字列に変換して表示
combined_geojson_str = json.dumps(combined_geojson, indent=2, ensure_ascii=False)
print(combined_geojson_str)

{
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "geometry": {
        "type": "Polygon",
        "coordinates": [
          [
            [
              139.6375,
              35.66666666666667
            ],
            [
              139.64999999999998,
              35.66666666666667
            ],
            [
              139.64999999999998,
              35.675000000000004
            ],
            [
              139.6375,
              35.675000000000004
            ],
            [
              139.6375,
              35.66666666666667
            ]
          ]
        ]
      },
      "properties": {
        "meshcode": "53394501"
      }
    },
    {
      "type": "Feature",
      "geometry": {
        "type": "Polygon",
        "coordinates": [
          [
            [
              139.65,
              35.66666666666667
            ],
            [
              139.6625,
              35.66666666666667
            ],
        

In [ ]:
# DataFrameの作成
features = combined_geojson['features']
df_2 = pd.DataFrame([
    {
        'meshcode': feature['properties']['meshcode'],
        'geometry': json.dumps(feature['geometry']),
        'color': feature['properties'].get('color', 'gray')  # 色プロパティがない場合は'gray'を使用
    }
    for feature in features
])

# カラースケールを定義 (RGBA)
colorscale = [
    [0, "rgba(0, 0, 255, 0.5)"],   # Blue with 50% opacity
    [0.5, "rgba(0, 255, 0, 0.5)"], # Green with 50% opacity
    [1, "rgba(255, 0, 0, 0.5)"]    # Red with 50% opacity
]

# PlotlyでGeoJSONを地図に描画
fig = px.choropleth_mapbox(
    df,
    geojson=combined_geojson,
    locations='meshcode',
    featureidkey='properties.meshcode',
    color='score',
    color_continuous_scale=colorscale,
    #color_discrete_map={"red": "red", "blue": "blue"},  # 色のマッピング（適宜調整）
    mapbox_style="carto-positron",
    center={"lat": 36.333333, "lon": 139.5},
    zoom=10,
    hover_name='address',
    hover_data=['score']
)



fig.update_layout(
    title_text="Multiple GeoJSON Polygons Visualization",
    geo=dict(
        scope='asia',
        projection_type='mercator'
    )
)

fig.show()